In [ ]:
import pandas as pd
import re
import geocoder
import pycountry

In [ ]:
df = pd.read_csv("IATA-original.csv")

In [ ]:
df

In [ ]:
df.dropna(how='any', inplace=True)
df.reset_index(inplace=True)
del df['index']
df

In [ ]:
df['IATA'] = df['IATA'].apply(lambda x: x.strip())

In [ ]:
def clean_IATA(iata):
    if len(iata) == 3:
        return iata
    print(iata)
    for i in range(10):
        if "[" + str(i) + "]" in iata:
            return iata.replace("[" + str(i) + "]", "")
    return iata

In [ ]:
df['IATA'] = df['IATA'].apply(clean_IATA)
df

In [ ]:
df['IATA-len'] = df['IATA'].apply(len)
df

In [ ]:
df.loc[df['IATA-len'] != 3]

In [ ]:
Beijing_index = df.loc[df['IATA-len'] != 3].index
Beijing_index

In [ ]:
df.loc[Beijing_index, 'IATA'] = "BJS"

In [ ]:
df.loc[df['IATA-len'] != 3]

In [ ]:
del df['IATA-len']

In [ ]:
df

In [ ]:
def clean_airport_name(airport):
    for i in range(10):
        if "[" + str(i) + "]" in airport:
            return airport.replace("[" + str(i) + "]", "")
    return airport

In [ ]:
df['Airport_name'] = df['Airport_name'].apply(clean_airport_name)
df

In [ ]:
def get_n_locations(location):
    if len(location.split(",")) > 3 or len(location.split(",")) == 1:
        print(location)
        return None
    return "marker"

In [ ]:
df['n_locations'] = df['Location'].apply(get_n_locations)

In [ ]:
df

In [ ]:
df.loc[df['n_locations'].isnull()]

In [ ]:
df.dropna(how="any", inplace=True)
df.reset_index(inplace=True)
del df['index']
del df['n_locations']

In [ ]:
df.loc[df['Location'].str.contains("[0-9]", regex=True)]

In [ ]:
def get_city(location):
    return location.split(",")[0]

In [ ]:
"Zagreb" in df['Location'].apply(get_city).tolist()

In [ ]:
df['City'] = df['Location'].apply(get_city)
df

In [ ]:
def get_state(location):
    if len(location.split(",")) == 2:
        return None
    else:
        return location.split(",")[1].replace("\n", " ").strip()


def get_country(location):
    if len(location.split(",")) == 2:
        country = location.split(",")[1]
        return re.sub(r'[0-9]+', '', country).replace("\n", " ").strip()
    else:
        country = location.split(",")[2]
        return re.sub(r'[0-9]+', '', country).replace("\n", " ").strip()

In [ ]:
#set(df['Location'].apply(get_state).tolist())

In [ ]:
#set(df['Location'].apply(get_country).tolist())

In [ ]:
df['State'] = df['Location'].apply(get_state)

In [ ]:
df['Country'] = df['Location'].apply(get_country)

In [ ]:
df

In [ ]:
len(df['State'].unique())

In [ ]:
len(df['Country'].unique())

In [ ]:
len(df['City'].unique())

In [ ]:
df.isnull().sum()

In [ ]:
def is_valid_IATA(iata):
    if re.search("[A-Z]{3}", iata):
        return True
    else:
        return False

In [ ]:
set(df['IATA'].apply(is_valid_IATA))

In [ ]:
del df['Location']

In [ ]:
df.head()

In [ ]:
def get_alpha_2(c):
    if pycountry.countries.get(name=c):
        return pycountry.countries.get(name=c).alpha_2
    if pycountry.countries.get(official_name=c):
        return pycountry.countries.get(official_name=c).alpha_2
    return None

In [ ]:
df['Country_alpha_2'] = df['Country'].apply(get_alpha_2)

In [ ]:
df

In [ ]:
def get_alpha_3(c):
    if pycountry.countries.get(name=c):
        return pycountry.countries.get(name=c).alpha_3
    if pycountry.countries.get(official_name=c):
        return pycountry.countries.get(official_name=c).alpha_3
    return None

In [ ]:
df['Country_alpha_3'] = df['Country'].apply(get_alpha_3)

In [ ]:
df

In [ ]:
df.loc[(df['Country_alpha_3'].isnull()) | (df['Country_alpha_2'].isnull())]

In [ ]:
len(df)

In [ ]:
set(df.loc[(df['Country_alpha_3'].isnull()) | (df['Country_alpha_2'].isnull())]['Country'])

In [ ]:
df['lat'] = None
df['lng'] = None

In [ ]:
df.columns

In [ ]:
def get_coordinates(rowitem, df):
    sep = ", "
    airport = sep.join([rowitem[1]['Airport_name'], rowitem[1]['City'], rowitem[1]['Country']] )
    #print(airport)
    g = geocoder.osm(rowitem[1]['IATA']) 
    if g:
        df.iloc[rowitem[0]]['lat'] = g.json['lat']
        df.iloc[rowitem[0]]['lng'] = g.json['lng']
        return
    g = geocoder.osm(rowitem[1]['Airport_name'])
    if g:
        df.iloc[rowitem[0]]['lat'] = g.json['lat']
        df.iloc[rowitem[0]]['lng'] = g.json['lng']
        return
    #g = geocoder.osm( sep.join([ rowitem[1]['Airport_name'], rowitem[1]['Country'] ])  )
    #if g:
    global c
    global unresolved
    unresolved.append(airport)
    c += 1
    print("None", c, airport)
    return None

In [ ]:
c = 0
unresolved = list()
for item in df.iterrows():
    get_coordinates(item, df)

In [ ]:
len(unresolved)

In [ ]:
unresolved

In [ ]:
df

In [ ]:
df.loc[ (df['lat'].isnull()) | (df['lng'].isnull()) ]

In [ ]:
df.to_csv('IATA-complete.csv', header=True, index=False)